## Load Packages

In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os

In [6]:
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import rasterio
import re
import rtree
import shapely
import pickle

ImportError: DLL load failed while importing _base: The specified module could not be found.

In [3]:
import progressbar # pip install progressbar2, not progressbar
from geopy.geocoders import Nominatim
from rasterio.windows import Window 
from tqdm import tqdm

In [ ]:
import data_eng.az_proc as ap

## Define Microsoft Azure Blob Root

In [5]:
# The(preferred) copy of NAIP in the West Europe Azure region

warnings.filterwarnings("ignore")
%matplotlib inline

## Load the spatial index of NAIP tiles

In [5]:
# Spatial index that maps lat/lon to NAIP tiles; we'll load this when we first 
# need to access it.
index = None

if index is None:
    index = ap.NAIPTileIndex()

Bypassing download of already-downloaded file tile_index.dat
Bypassing download of already-downloaded file tile_index.idx
Bypassing download of already-downloaded file tiles.p


In [6]:
arr = np.empty([0, 2])
arr.shape

(0, 2)

In [67]:
def lons_lat_to_filepaths(lons, lats, index):
    """
    Calculate file paths given lat and lat
    """
    all_paths = np.empty(shape=(1,8))
    for i in tqdm(range(len(lons))):
        naip_file_pathways = index.lookup_tile(lats[i], lons[i])
        if naip_file_pathways != None:
            select_path = []
            for ii in range(len(naip_file_pathways)):
                tmp = naip_file_pathways[ii].split('/')
                tmp = np.hstack((tmp, naip_file_pathways[ii].split('/')[3].split("_")[1]))
                iii = iter(tmp[5].split("_",4))
                tmp = np.hstack((tmp, list((map("_".join,zip(*[iii]*4)) ))))
                select_path.append(tmp)
            select_path = np.array(select_path)
            select_path = select_path[select_path[:,2] >= "2018"] #filter out years to get the most recent data that will include the highest resolution data

            select_path = select_path[(select_path[:,6] == "60cm") | (select_path[:,6] == "060cm")] #select only pathways with 60cm
                        
            all_paths = np.vstack((all_paths, select_path)) #add to the rest of the paths
            
    file_pathways = np.delete(all_paths, 0, axis=0)

    file_pathways = np.unique(file_pathways, axis=0) #select unique values
    return file_pathways

In [69]:
expanded_sites = pd.read_csv("image_download_azure/identified_sites.csv") #read in sheet of quadrangles
expanded_sites_lat = expanded_sites["Lat"].tolist()
expanded_sites_lon = expanded_sites["Lon"].tolist()
assert len(expanded_sites_lat) == len(expanded_sites_lat)
print(len(expanded_sites_lon))
len(lons_lat_to_filepaths(expanded_sites_lon, expanded_sites_lat, index))

100%|██████████| 65/65 [00:00<00:00, 1354.11it/s]

65
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections


35

## EIA and HFID Petroleum Data Sources

## Define filepathways to save data 

In [71]:
#set folder and specify destination
natech_dir = os.path.join('/shared_space','natech') #use the shared_space folder as the base because there is ample storage capacilty 
os.makedirs(natech_dir,exist_ok=True)

naip_dir = os.path.join(natech_dir,'naip') #directory for the naip data in the base (csr33) directory 
os.makedirs(naip_dir,exist_ok=True)

naip_reshape_dir = os.path.join(natech_dir,'reshape') #directory to hold reshaped naip images
os.makedirs(naip_reshape_dir,exist_ok=True)

naip_chips_dir = os.path.join(natech_dir,'chips') #directory to hold reshaped naip images
os.makedirs(naip_chips_dir,exist_ok=True)

# EIA and HFID 

### Load Data

Homeland Infrastructure Foundation-Level Data (HIFLD) - Petroleum Terminals

https://hifld-geoplatform.opendata.arcgis.com/datasets/7841aba67178425cbf33995fc914e2fe_0/data

In [74]:
hfid_petroleum_terminals = pd.read_csv("image_download_azure/Petroleum_Terminals_HFID.csv") #read in sheet of quadrangles
hfid_lons = hfid_petroleum_terminals["X"].tolist()
hfid_lats = hfid_petroleum_terminals["Y"].tolist()

EIA - Strategic Petroleum Reserves

https://atlas.eia.gov/datasets/strategic-petroleum-reserves?geometry=-159.521%2C0.792%2C-28.212%2C52.750

In [75]:
eia_strategic_petroleum_reserves = pd.read_csv("image_download_azure/Strategic_Petroleum_Reserves.csv") #read in sheet of quadrangles
eia_spr_lons = eia_strategic_petroleum_reserves["X"].tolist()
eia_spr_lats = eia_strategic_petroleum_reserves["Y"].tolist()

EIA - Petroleum Product Terminals

https://atlas.eia.gov/datasets/petroleum-product-terminals 

In [76]:
eia_petroleum_product_terminals = pd.read_csv("image_download_azure/Petroleum_Product_Terminals.csv") #read in sheet of quadrangles
eia_ppt_lons = eia_petroleum_product_terminals["X"].tolist()
eia_ppt_lats = eia_petroleum_product_terminals["Y"].tolist()

EIA - Northeast Petroleum Reserves

https://atlas.eia.gov/datasets/northeast-petroleum-reserves 

In [77]:
eia_northeast_petroleum_reserves = pd.read_csv("image_download_azure/Northeast_Petroleum_Reserves.csv") #read in sheet of quadrangles
eia_npr_lons = eia_northeast_petroleum_reserves["X"].tolist()
eia_npr_lats = eia_northeast_petroleum_reserves["Y"].tolist()

EIA - Petroleum Refineries

https://atlas.eia.gov/datasets/petroleum-refineries?geometry=-13.914%2C-56.555%2C151.320%2C84.803

In [78]:
eia_petroleum_refineries = pd.read_csv("image_download_azure/Petroleum_Refineries.csv") #read in sheet of quadrangles
eia_pr_lons = eia_petroleum_refineries["X"].tolist()
eia_pr_lats = eia_petroleum_refineries["Y"].tolist()

EIA - Natural Gas Processing Plants

https://atlas.eia.gov/datasets/natural-gas-processing-plants

In [79]:
eia_natural_gas_processing_plants = pd.read_csv("image_download_azure/Natural_Gas_Processing_Plants.csv") #read in sheet of quadrangles
eia_ngpp_lons = eia_natural_gas_processing_plants["X"].tolist()
eia_ngpp_lats = eia_natural_gas_processing_plants["Y"].tolist()

### Get the Filepathways, tile name, tile URL for EIA HFID Data

In [84]:
hfid_file_pathways = lons_lat_to_filepaths(hfid_lons, hfid_lats, index)
eia_spr_file_pathways = lons_lat_to_filepaths(eia_spr_lons, eia_spr_lats, index)
eia_ppt_file_pathways = lons_lat_to_filepaths(eia_ppt_lons, eia_ppt_lats, index)
eia_npr_file_pathways = lons_lat_to_filepaths(eia_npr_lons, eia_npr_lats, index)
eia_pr_file_pathways = lons_lat_to_filepaths(eia_pr_lons, eia_pr_lats, index)
eia_ngpp_file_pathways = lons_lat_to_filepaths(eia_ngpp_lons, eia_ngpp_lats, index)
expanded_file_pathways = lons_lat_to_filepaths(expanded_sites_lon, expanded_sites_lat, index)

#filepathways 
petrol_file_pathways = np.vstack((hfid_file_pathways, eia_spr_file_pathways, eia_ppt_file_pathways, eia_npr_file_pathways,
                                  eia_pr_file_pathways, eia_ngpp_file_pathways, expanded_file_pathways)) #combine filepaths from multiple sources

petrol_file_pathways = np.unique(petrol_file_pathways, axis=0) remove duplicates

 11%|█         | 251/2338 [00:00<00:02, 990.24it/s] 

No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile in

 28%|██▊       | 644/2338 [00:00<00:02, 782.27it/s] 

No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections


  6%|▌         | 85/1476 [00:00<00:01, 849.96it/s]

...


No tile intersections
No tile intersections


 46%|████▌     | 62/135 [00:00<00:00, 619.98it/s]

No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections


100%|██████████| 65/65 [00:00<00:00, 1477.21it/s]

No tile intersections
No tile intersections
No tile intersections
No tile intersections
No tile intersections


In [93]:
#tile names and urls 
tile_name_eia_hfid_expanded, tile_url_eia_hfid_expanded = ap.filepaths_to_tile_name_tile_url(petrol_file_pathways)
tile_name_tile_url_eia_hfid_expanded = np.column_stack((tile_name_eia_hfid_expanded, tile_url_eia_hfid_expanded))

In [94]:
tile_name_tile_url_eia_hfid_expanded.shape

(2457, 2)

# Group Identified ASTs

## Thirty Ports

In [95]:
thirty_port_quads = pd.read_csv("image_download_azure/Quadrangles_of_interest.csv") #read in sheet of quadrangles

tile_name_thirty_ports, tile_url_thirty_ports = ap.collected_quads_to_tile_name_tile_url(thirty_port_quads) # identify filespaths/urls for quads of interest

tile_name_tile_url_thirty_ports = np.column_stack((tile_name_thirty_ports, tile_url_thirty_ports))

# Combine Filepaths from each source

In [96]:
tile_name_tile_url_eia_hfid_expanded_thirty_ports = np.vstack((tile_name_tile_url_eia_hfid_expanded, tile_name_tile_url_thirty_ports))
print(tile_name_tile_url_eia_hfid_expanded_thirty_ports.shape)
tile_name_tile_url_eia_hfid_expanded_thirty_ports = np.unique(tile_name_tile_url_eia_hfid_expanded_thirty_ports, axis=0) #remove duplicates
print(tile_name_tile_url_eia_hfid_expanded_thirty_ports.shape)

(2605, 2)
(2535, 2)


# Save tile name and url

In [113]:
# save array Get array of the expanded data
np.save("image_download_azure/tile_name_tile_url_eia_hfid_thirty_ports_expanded", tile_name_tile_url_eia_hfid_expanded_thirty_ports)

In [114]:
tile_name_tile_url_eia_hfid_thirty_ports = np.load('image_download_azure/tile_name_tile_url_eia_hfid_thirty_ports.npy')
dif = np.array(list(set(map(tuple, tile_name_tile_url_eia_hfid_expanded_thirty_ports)) - set(map(tuple, tile_name_tile_url_eia_hfid_thirty_ports ))))
#save only the tiles that were not originally included in the assessment set
np.save("image_download_azure/tile_name_tile_url_expanded_only", dif)

In [5]:
expanded_full = np.load("image_download_azure/tile_name_tile_url_eia_hfid_thirty_ports_expanded.npy")
expanded_only_additional = np.load("image_download_azure/tile_name_tile_url_expanded_only.npy")
labeled_first_set = np.load("tile_name_tile_url_labeled.npy")
#labeled_first_set_remaining = np.load("tile_name_tile_url_remaining.npy")

In [25]:
print(expanded_only_additional.shape)
print(labeled_first_set.shape)
print(labeled_first_set_remaining.shape)

['https://naipblobs.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2019/24081/m_2408126_ne_17_060_20191128.tif'
 'https://naipblobs.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2019/24081/m_2408126_se_17_060_20191128.tif'
 'https://naipblobs.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2019/25080/m_2508014_se_17_060_20191126.tif'
 ...
 'https://naipblobs.blob.core.windows.net/naip/v002/wa/2019/wa_60cm_2019/48122/m_4812237_ne_10_060_20190806.tif'
 'https://naipblobs.blob.core.windows.net/naip/v002/wa/2019/wa_60cm_2019/48122/m_4812258_se_10_060_20191011.tif'
 'https://naipblobs.blob.core.windows.net/naip/v002/wa/2019/wa_60cm_2019/48122/m_4812258_sw_10_060_20191029.tif']
(1151, 2)
(1390, 2)
(0, 2)


In [39]:
states = []
for tile in expanded_full[:,1]:
    states.append(tile.split("/")[5])
print(np.unique(np.array(states)))

['al' 'ar' 'az' 'ca' 'co' 'ct' 'de' 'fl' 'ga' 'ia' 'id' 'il' 'in' 'ks'
 'ky' 'la' 'ma' 'md' 'me' 'mi' 'mn' 'mo' 'ms' 'mt' 'nc' 'nd' 'ne' 'nh'
 'nj' 'nm' 'nv' 'ny' 'oh' 'ok' 'pa' 'ri' 'sc' 'sd' 'tn' 'tx' 'ut' 'va'
 'vt' 'wa' 'wi' 'wv' 'wy']
